### 1. 利用python语言编写Apriori算法，挖掘出下表中数据集的频繁项集，最小支持度为30%。

In [3]:
from numpy import *

def loadDataSet():
    return  [ ['a', 'b', 'd', 'e'],
                ['b', 'c', 'd'],
                ['a', 'b', 'd', 'e'],
                ['a', 'c', 'd', 'e'],
                ['b', 'b', 'd', 'e'],
                ['b', 'd', 'e'],
                [ 'c', 'd'],
                ['a', 'b', 'c'] ,
                ['a', 'd', 'e'],
                ['b', 'd',]]
 

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    
    return list(map(frozenset, C1))  
 


def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):   
                if can not in ssCnt:    
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []        
    supportData = {}    
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData 
 


def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk): 
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]  
            L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()        
            if L1==L2:
                retList.append(Lk[i] | Lk[j]) 
    return retList  
 


def apriori(dataSet, minSupport = 0.3):
    D = list(map(set, dataSet)) 
            
    C1 = createC1(dataSet)      
    L1, supportData = scanD(D, C1, minSupport)  
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):    
        Ck = aprioriGen(L[k-2], k)  
        Lk, supK = scanD(D, Ck, minSupport) 
        supportData.update(supK)    
        L.append(Lk)
        k += 1  
    return L, supportData
 
dataSet = loadDataSet()
L,suppData = apriori(dataSet)
print(L)
print(suppData)


[[frozenset({'c'}), frozenset({'e'}), frozenset({'d'}), frozenset({'b'}), frozenset({'a'})], [frozenset({'c', 'd'}), frozenset({'b', 'a'}), frozenset({'a', 'd'}), frozenset({'b', 'd'}), frozenset({'a', 'e'}), frozenset({'b', 'e'}), frozenset({'d', 'e'})], [frozenset({'e', 'b', 'd'}), frozenset({'e', 'a', 'd'})], []]
{frozenset({'a'}): 0.5, frozenset({'b'}): 0.7, frozenset({'d'}): 0.9, frozenset({'e'}): 0.6, frozenset({'c'}): 0.4, frozenset({'d', 'e'}): 0.6, frozenset({'b', 'e'}): 0.4, frozenset({'a', 'e'}): 0.4, frozenset({'b', 'd'}): 0.6, frozenset({'a', 'd'}): 0.4, frozenset({'b', 'a'}): 0.3, frozenset({'c', 'd'}): 0.3, frozenset({'c', 'b'}): 0.2, frozenset({'c', 'e'}): 0.1, frozenset({'c', 'a'}): 0.2, frozenset({'b', 'a', 'd'}): 0.2, frozenset({'b', 'a', 'e'}): 0.2, frozenset({'e', 'a', 'd'}): 0.4, frozenset({'e', 'b', 'd'}): 0.4}


### 2. （选做）利用python语言编写FP增长算法，挖掘出上表中数据集的频繁项集，最小支持度为30%。

In [4]:
def loadSimpDat():
    simpDat = [ ['a', 'b', 'd', 'e'],
                ['b', 'c', 'd'],
                ['a', 'b', 'd', 'e'],
                ['a', 'c', 'd', 'e'],
                ['b', 'b', 'd', 'e'],
                ['b', 'd', 'e'],
                [ 'c', 'd'],
                ['a', 'b', 'c'] ,
                ['a', 'd', 'e'],
                ['b', 'd',]]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        fset = frozenset(trans)
        retDict.setdefault(fset, 0)
        retDict[fset] += 1
    return retDict

class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print('   ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)


def createTree(dataSet, minSup=1):
    headerTable = {}
    
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + 1

    
    lessThanMinsup = list(filter(lambda k:headerTable[k] < minSup, headerTable.keys()))
    for k in lessThanMinsup: del(headerTable[k])

    freqItemSet = set(headerTable.keys())
    
    if len(freqItemSet) == 0:
        return None, None

    for k in headerTable:
        headerTable[k] = [headerTable[k], None]

    retTree = treeNode('φ', 1, None)
    
    for tranSet, count in dataSet.items():
        
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]

        if len(localD) > 0:
            
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: (p[1],p[0]), reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable


def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  
        inTree.children[items[0]].inc(count)  
    else:  
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:  
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:  
        updateTree(items[1:], inTree.children[items[0]], headerTable, count)


def updateHeader(nodeToTest, targetNode):  
    while (nodeToTest.nodeLink != None):  
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

simpDat = loadSimpDat()
dictDat = createInitSet(simpDat)
myFPTree,myheader = createTree(dictDat, 3)
myFPTree.disp()

    φ   1
       d   9
          b   6
             e   4
                a   2
             c   1
          e   2
             c   1
                a   1
             a   1
          c   1
       b   1
          c   1
             a   1


### 实验总结
1. 学习了Apriori算法,它利用逐层搜索的迭代方法找出数据库中项集的关系，以形成规则，其过程由连接（类矩阵运算）与剪枝（去掉那些没必要的中间结果）组成。
2. Apriori算法采用了逐层搜索的迭代的方法，算法简单明了，没有复杂的理论推导，也易于实现。
3. Apriori算法会产生大量的中间项集, 同时算法适应面较窄
4. 构造出条件Fp树便于选取频繁项集